This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [13]:
# Lib imports
import os
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras import regularizers, callbacks
import numpy as np

In [14]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = 'C:/Users/heral/Downloads/Train'  # e.g. './muffin-vs-chihuahua/train'
test_dir = 'C:/Users/heral/Downloads/Test'    # e.g. './muffin-vs-chihuahua/test'

In [15]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [16]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [17]:
# SIMPLE CNN MODEL ARCHITECTURE (IMPROVED)
import random
# reproducibility
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

# Regularization and dropout settings
weight_decay = 1e-4

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', kernel_regularizer=regularizers.l2(weight_decay), input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3,3), activation='relu', kernel_regularizer=regularizers.l2(weight_decay)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3,3), activation='relu', kernel_regularizer=regularizers.l2(weight_decay)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

In [18]:
# optimizer with learning rate schedule + compile
initial_learning_rate = 1e-3
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps=10000, decay_rate=0.9, staircase=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,306,177 (12.61 MB)

 Trainable params: 3,305,473 (12.61 MB)

 Non-trainable params: 704 (2.75 KB)

In [19]:
# TRAINING THE CNN (IMPROVED)
from tensorflow.keras import callbacks as cb

es = cb.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
rlp = cb.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

epochs = 10
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
    callbacks=[es, rlp]
)

# Evaluate on test set and print accuracy
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}")

# Save improved model with required filename
model.save('exercise_6_trained_model_improved.h5')
print('Saved model to exercise_6_trained_model_improved.h5')

Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 41s 325ms/step - accuracy: 0.7846 - loss: 0.5938 - val_accuracy: 0.5407 - val_loss: 3.5478 - learning_rate: 0.0010
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 41s 325ms/step - accuracy: 0.7846 - loss: 0.5938 - val_accuracy: 0.5407 - val_loss: 3.5478 - learning_rate: 0.0010
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 40s 339ms/step - accuracy: 0.8266 - loss: 0.4857 - val_accuracy: 0.5407 - val_loss: 2.5826 - learning_rate: 0.0010
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 40s 339ms/step - accuracy: 0.8266 - loss: 0.4857 - val_accuracy: 0.5407 - val_loss: 2.5826 - learning_rate: 0.0010
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 40s 337ms/step - accuracy: 0.8506 - loss: 0.4261 - val_accuracy: 0.5407 - val_loss: 1.8130 - learning_rate: 0.0010
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 40s 337ms/step - accuracy: 0.8506 - loss: 0.4261 - val_accuracy: 0.5407 - val_loss: 1.8130 - learning_rate: 0.0010
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 40s 338ms/step - accuracy: 0.8

Test Accuracy: 0.9113
Saved model to exercise_6_trained_model_improved.h5
Saved model to exercise_6_trained_model_improved.h5


In [20]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 121ms/step - accuracy: 0.9113 - loss: 0.3069
Test Accuracy: 0.9113175868988037
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 121ms/step - accuracy: 0.9113 - loss: 0.3069
Test Accuracy: 0.9113175868988037


In [21]:
model.save('exercise_6_trained_model_improved.h5')

In [22]:
# SIMPLE INFERENCE SCRIPT (returns confidence and can append to CSV)
from tensorflow.keras.preprocessing import image
import csv
import os

def predict_image(img_path, model_path='exercise_6_trained_model_improved.h5', write_csv=None):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = float(model.predict(img_array, verbose=0)[0,0])
    label = "Muffin" if pred >= 0.5 else "Chihuahua"
    print(f"{os.path.basename(img_path)} -> Prediction: {label} (confidence: {pred:.4f})")
    if write_csv:
        header_needed = not os.path.exists(write_csv) or os.path.getsize(write_csv) == 0
        with open(write_csv, 'a', newline='') as f:
            writer = csv.writer(f)
            if header_needed:
                writer.writerow(['image','label','confidence'])
            writer.writerow([os.path.basename(img_path), label, pred])
    return label, pred


In [23]:
# Example usage and capture Run predictions
image_paths = [
    'C:/Users/heral/Downloads/muff.jpg',
    'C:/Users/heral/Downloads/chi.jpg',
    'C:/Users/heral/Downloads/muff2.jpg',
    'C:/Users/heral/Downloads/chi2.jpg',
    'C:/Users/heral/Downloads/muff3.jpg'
]
run_csv = 'run_predictions.csv'
for p in image_paths:
    try:
        predict_image(p, model_path='exercise_6_trained_model_improved.h5', write_csv=run_csv)
    except Exception as e:
        print('Error predicting', p, e)

muff.jpg -> Prediction: Muffin (confidence: 0.8135)


chi.jpg -> Prediction: Chihuahua (confidence: 0.0004)


muff2.jpg -> Prediction: Muffin (confidence: 0.9967)


chi2.jpg -> Prediction: Muffin (confidence: 0.5538)
muff3.jpg -> Prediction: Muffin (confidence: 0.9912)
muff3.jpg -> Prediction: Muffin (confidence: 0.9912)
